In [1]:
import pandas as pd
import numpy as np
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', None)

In [2]:
# read csv data as dataframe
data = pd.read_csv('/home/iik/Desktop/tiria/data/2021-07-20_21-57_results.csv', delimiter=';')
data = data.rename(columns={'Schueler-ID': 'child_id'})
data.head()

,Klasse,Stufe,Lehrer,ID,child_id,Test,Aufgabe,Gewaehlt1,Gewaehlt2,Richtig,Ergebnis,Start,Ende,Dauer,GesamtRichtig,GesamtFalsch,Level,GesamtLevel
0,1a,1,Müller,101,1,1,Trial 1,Büffel,Delfin,"Dachs,Delfin",False,07.06.2021 17:36:30,07.06.2021 17:37:24,00:00:54,8,2,C,C
1,1a,1,Müller,101,1,1,Trial 2,Wal,Tiger,"Wildschwein,Wal",False,07.06.2021 17:36:30,07.06.2021 17:37:24,00:00:54,8,2,C,C
2,1a,1,Müller,101,1,1,1,Ameise,Fuchs,"Ameise,Adler",False,07.06.2021 17:36:30,07.06.2021 17:37:24,00:00:54,8,2,C,C
3,1a,1,Müller,101,1,1,2,Baum,Biene,"Baum,Biene",True,07.06.2021 17:36:30,07.06.2021 17:37:24,00:00:54,8,2,C,C
4,1a,1,Müller,101,1,1,3,Erdbeere,Ente,"Erdbeere,Ente",True,07.06.2021 17:36:30,07.06.2021 17:37:24,00:00:54,8,2,C,C


In [3]:
# rename variables
data = data.rename(columns={'Schueler-ID': 'child_id', 
                            'Gewaehlt1': 'choice_1',
                            'Gewaehlt2':'choice_2',
                            'GesamtRichtig':'Gesamt_Richtig',
                            'GesamtFalsch':'Gesamt_Falsch',
                            'GesamtLevel': 'Gesamt_Level'})

# delete the tutorial tasks from df (trial 1 & trial 2) as well as test trials (which is ID == 101 and ==202) as they should not be included in the analysis
data = data.loc[~((data['Aufgabe']=='Trial 1') | (data['Aufgabe']=='Trial 2'))]

# delete all test trials 
data = data.loc[~((data.ID ==101) | (data.ID ==202))]

# transform datatype from variable 'Aufgabe' to type integer
data.Aufgabe = data.Aufgabe.astype(int)

# Create Date time object
data.Start = pd.to_datetime(data.Start, format='%d.%m.%Y %H:%M:%S')
data.Ende = pd.to_datetime(data.Ende, format='%d.%m.%Y %H:%M:%S')

data.insert(0, 'unique_id', data.ID.astype(str)+'_'+ data.child_id.astype(str))

data['Ergebnis'] = data['Ergebnis'].replace({'True':1, 'False':0, 'Keine Auswahl': 99})
data['Ergebnis'] = data['Ergebnis'].astype(int)

data.head()

,unique_id,Klasse,Stufe,Lehrer,ID,child_id,Test,Aufgabe,choice_1,choice_2,Richtig,Ergebnis,Start,Ende,Dauer,Gesamt_Richtig,Gesamt_Falsch,Level,Gesamt_Level
1832,303_0,Fuchsbande,2,Cordula Wuttke,303,0,1,1,Ameise,Adler,"Ameise,Adler",1,2021-06-29 11:10:05,2021-06-29 11:11:08,00:01:02,10,0,C,B
1833,303_0,Fuchsbande,2,Cordula Wuttke,303,0,1,2,Baum,Biene,"Baum,Biene",1,2021-06-29 11:10:05,2021-06-29 11:11:08,00:01:02,10,0,C,B
1834,303_0,Fuchsbande,2,Cordula Wuttke,303,0,1,3,Erdbeere,Ente,"Erdbeere,Ente",1,2021-06-29 11:10:05,2021-06-29 11:11:08,00:01:02,10,0,C,B
1835,303_0,Fuchsbande,2,Cordula Wuttke,303,0,1,4,Fisch,Fliege,"Fisch,Fliege",1,2021-06-29 11:10:05,2021-06-29 11:11:08,00:01:02,10,0,C,B
1836,303_0,Fuchsbande,2,Cordula Wuttke,303,0,1,5,Gras,Giraffe,"Gras,Giraffe",1,2021-06-29 11:10:05,2021-06-29 11:11:08,00:01:02,10,0,C,B


In [4]:
data['idx'] = data.groupby('unique_id').cumcount()
data = data.pivot_table(index=['Test', 'Stufe', "unique_id", 'Level', 'Gesamt_Level'], 
                    columns='Aufgabe', 
                    values='Ergebnis').reset_index()

data = pd.DataFrame(data.to_records())
data = data.drop(columns=['index'])
data.head()

,Test,Stufe,unique_id,Level,Gesamt_Level,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50
0,1,1,404_0,C,C,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,404_1,A,A,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,1,404_10,B,B,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,1,404_11,A,A,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,1,404_12,B,C,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# drop dublicates


In [6]:
table_out = data.to_csv('/home/iik/Desktop/tiria/data/allTests.csv', index=False)
print('ok')

ok


In [7]:
dict_test_df={}
test_name = data.Test.unique()
for test in test_name:
    df_test = data.loc[data['Test'] == test]
    dict_test_df[test]= df_test

In [8]:
listTest=[1,2,3,4]
for t in listTest:
    test = dict_test_df.get(t)
    filepath = '/home/iik/Desktop/tiria/data/test_'+ str(t)+'.csv'
    test_out = test.to_csv(filepath)